In [259]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [260]:
df = pd.read_csv('df_final.csv')
df = df.drop('Unnamed: 0',axis=1)
df

,ehr,birth_date,diagnosis_date,death_date,age,er,her2,ki67,pr,pregnancy,...,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,10011773,05-07-1959,02-04-2015,NaN,65,0.0,0.0,19.0,1.0,0.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,0,SIN TRATAMIENTO,1.0,0.0,0.0,0.0
1,10020495,02-10-1953,04-12-2017,NaN,71,1.0,0.0,9.0,0.0,0.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0
2,10030299,16-08-1966,27-06-2019,NaN,58,1.0,0.0,18.0,1.0,3.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0
3,10030824,03-03-1953,07-09-2018,NaN,71,1.0,0.0,18.0,1.0,0.0,...,0.0,2,2.0,0.0,IIIA,IIIA,3.0,0.0,1.0,1.0
4,10041592,06-07-1959,19-11-2018,NaN,65,0.0,0.0,65.0,0.0,0.0,...,0.0,4,0.0,0.0,IB,IA,2.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99343880,23-08-1944,28-05-2018,NaN,80,1.0,0.0,68.0,1.0,2.0,...,1.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0
11163,99443402,17-06-1959,21-01-2018,NaN,65,1.0,0.0,22.0,0.0,0.0,...,1.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0
11164,99656792,26-03-1976,23-07-2017,NaN,48,1.0,1.0,49.0,0.0,0.0,...,0.0,4,0.0,0,IA,SIN TRATAMIENTO,1.0,0.0,0.0,1.0
11165,99690760,21-07-1961,28-10-2013,NaN,63,1.0,0.0,20.0,1.0,1.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0


In [261]:
correlaciones = df.corr(method='pearson')
correlaciones.style.background_gradient(cmap='coolwarm',axis=None).set_precision(3)

C:\Users\34699\AppData\Local\Temp\ipykernel_2808\732683472.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlaciones = df.corr(method='pearson')
C:\Users\34699\AppData\Local\Temp\ipykernel_2808\732683472.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  correlaciones.style.background_gradient(cmap='coolwarm',axis=None).set_precision(3)


,ehr,age,er,her2,ki67,pr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age,n_tumor,n_category,m_category,grade,ductal,lobular,neoadjuvant
ehr,1.000,-0.025,-0.001,0.019,-0.020,-0.007,0.002,-0.004,0.005,0.005,0.017,0.022,0.004,-0.005,-0.012,0.022,-0.006,-0.005,-0.007
age,-0.025,1.000,0.008,0.001,-0.012,-0.016,0.007,0.008,0.005,-0.005,0.016,0.051,0.001,-0.017,0.003,-0.012,-0.015,0.012,-0.017
er,-0.001,0.008,1.000,-0.049,-0.156,0.116,-0.020,-0.017,0.008,-0.014,-0.020,-0.005,0.008,0.004,0.006,-0.039,-0.006,0.009,-0.051
her2,0.019,0.001,-0.049,1.000,0.178,-0.074,0.005,-0.012,0.023,0.002,-0.003,0.011,-0.001,0.008,-0.010,0.048,0.004,0.005,0.128
ki67,-0.020,-0.012,-0.156,0.178,1.000,-0.153,0.014,0.002,-0.004,0.023,0.010,0.000,0.006,-0.007,0.001,0.033,-0.015,0.005,0.075
pr,-0.007,-0.016,0.116,-0.074,-0.153,1.000,0.006,0.008,0.017,-0.019,0.003,-0.002,-0.008,0.013,-0.019,-0.050,0.013,-0.001,-0.048
pregnancy,0.002,0.007,-0.020,0.005,0.014,0.006,1.000,0.693,0.264,0.316,-0.004,-0.004,-0.011,0.011,0.001,-0.003,0.011,0.022,0.014
birth,-0.004,0.008,-0.017,-0.012,0.002,0.008,0.693,1.000,-0.223,-0.136,-0.001,-0.008,-0.006,0.007,0.003,-0.011,0.011,0.008,0.000
caesarean,0.005,0.005,0.008,0.023,-0.004,0.017,0.264,-0.223,1.000,-0.273,-0.014,0.005,-0.006,0.010,-0.008,0.001,0.003,0.012,0.010
abort,0.005,-0.005,-0.014,0.002,0.023,-0.019,0.316,-0.136,-0.273,1.000,0.009,-0.000,-0.002,-0.002,0.006,0.010,-0.002,0.012,0.013


Observamos que las correlaciones no nos aportan información, ya que son muy bajas para todos los valores no esperados. Para las variables en las que sí esperábamos correlación alta, como 'birth' y 'pregnancy', o 'ductal' y 'lobular' sí que se da dicha correlación, pero no nos aporta iformación nueva.

In [262]:
import tensorflow as tf 
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [263]:
datos_modelo = df.copy()
datos_modelo = datos_modelo.drop(['ehr','death_date','diagnosis_date','t_category','n_category','m_category','t_category_after_neoadj',
       'n_category_after_neoadj', 'm_category_after_neoadj'],axis=1)
datos_modelo.columns

Index(['birth_date', 'age', 'er', 'her2', 'ki67', 'pr', 'pregnancy', 'birth',
       'caesarean', 'abort', 'menarche_age', 'menopause_age', 'n_tumor',
       'stage_diagnosis', 'stage_after_neo', 'grade', 'ductal', 'lobular',
       'neoadjuvant'],
      dtype='object')

In [264]:
fechas = datos_modelo['birth_date'].values
edades = fechas.copy()
for i in range(len(fechas)):
    edades[i] = int(2024 - int(datos_modelo['birth_date'][i][6:]))
datos_modelo['age'] = edades
datos_modelo['age'].astype('int')
datos_modelo = datos_modelo.drop('birth_date',axis=1)
datos_modelo.dtypes

age                 object
er                 float64
her2               float64
ki67               float64
pr                 float64
pregnancy          float64
birth              float64
caesarean          float64
abort              float64
menarche_age       float64
menopause_age      float64
n_tumor            float64
stage_diagnosis     object
stage_after_neo     object
grade              float64
ductal             float64
lobular            float64
neoadjuvant        float64
dtype: object

In [265]:
datos_modelo['menarche_age'] = datos_modelo['menarche_age'].astype('int')
datos_modelo.dtypes

age                 object
er                 float64
her2               float64
ki67               float64
pr                 float64
pregnancy          float64
birth              float64
caesarean          float64
abort              float64
menarche_age         int32
menopause_age      float64
n_tumor            float64
stage_diagnosis     object
stage_after_neo     object
grade              float64
ductal             float64
lobular            float64
neoadjuvant        float64
dtype: object

In [266]:
pacientes_no_quimio = datos_modelo[datos_modelo['neoadjuvant']==0]
pacientes_no_quimio

,age,er,her2,ki67,pr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age,n_tumor,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,65,0.0,0.0,19.0,1.0,0.0,0.0,0.0,0.0,13,49.0,1.0,0,SIN TRATAMIENTO,1.0,0.0,0.0,0.0
1,71,1.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,13,49.0,1.0,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0
2,58,1.0,0.0,18.0,1.0,3.0,3.0,0.0,0.0,13,49.0,1.0,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0
5,67,1.0,0.0,18.0,1.0,2.0,1.0,0.0,1.0,13,49.0,1.0,IV,SIN TRATAMIENTO,3.0,1.0,0.0,0.0
11,67,1.0,0.0,14.0,0.0,2.0,0.0,0.0,2.0,18,44.0,1.0,IA,SIN TRATAMIENTO,3.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11158,67,1.0,0.0,29.0,0.0,1.0,1.0,0.0,0.0,13,49.0,2.0,IA,SIN TRATAMIENTO,3.0,1.0,0.0,0.0
11160,66,1.0,0.0,52.0,1.0,0.0,0.0,0.0,0.0,13,49.0,2.0,IIA,SIN TRATAMIENTO,1.0,0.0,0.0,0.0
11162,80,1.0,0.0,68.0,1.0,2.0,2.0,0.0,0.0,16,47.0,2.0,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0
11163,65,1.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,13,49.0,2.0,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0


In [267]:
datos_modelo = datos_modelo.replace("SIN TRATAMIENTO",np.nan)
datos_modelo = datos_modelo.dropna()
datos_modelo.dtypes

age                  int64
er                 float64
her2               float64
ki67               float64
pr                 float64
pregnancy          float64
birth              float64
caesarean          float64
abort              float64
menarche_age         int32
menopause_age      float64
n_tumor            float64
stage_diagnosis     object
stage_after_neo     object
grade              float64
ductal             float64
lobular            float64
neoadjuvant        float64
dtype: object

In [268]:
pacientes_quimio = datos_modelo[datos_modelo['neoadjuvant']==1]
pacientes_quimio.iloc[1,:].values

array([65, 0.0, 0.0, 65.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13, 49.0, 1.0, 'IB',
       'IA', 2.0, 1.0, 0.0, 1.0], dtype=object)

In [269]:
dic_estadios = {'0':0,'IA':1,'IB':2,'IIA':3,'IIB':4,'IIIA':5,'IIIB':6,'IIIC':7,'IV':8}
pacientes_quimio = pacientes_quimio.replace(dic_estadios)
target = pacientes_quimio['stage_after_neo'].values
pacientes_quimio = pacientes_quimio.drop('stage_after_neo',axis=1)
x_train = pacientes_quimio.values[:5000]
x_test = pacientes_quimio.values[5000:]

In [283]:
y_train = np.zeros((5000,9))
for i in range(5000):
    y_train[i][target[i]] = 1
y_test = target[5000:]
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [271]:
scaler = StandardScaler()
datos_normalizados = scaler.fit_transform(x_train)

In [272]:
regresion = LogisticRegression()
red_neuronal = models.Sequential([
    layers.Input(shape=(17)),
    layers.Dense(32,activation='relu'),
    layers.Dropout(rate=0.25),
    layers.Dense(128,activation='relu'),
    layers.Dense(9,activation='sigmoid')
])

In [279]:
regresion.fit(x_train,target[:5000])
predicciones_reg = regresion.predict(x_test)
precision = accuracy_score(y_test, predicciones_reg)
print(f'Precision regresion: {precision}')

Precision regresion: 0.34229390681003585


c:\Users\34699\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [284]:
red_neuronal.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
red_neuronal.fit(x_train, y_train, epochs=50)

Epoch 1/50
157/157 [==============================] - 1s 3ms/step - loss: 2.8053 - accuracy: 0.2514
Epoch 2/50
157/157 [==============================] - 0s 3ms/step - loss: 1.9989 - accuracy: 0.3098
Epoch 3/50
157/157 [==============================] - 1s 3ms/step - loss: 1.9006 - accuracy: 0.3428
Epoch 4/50
157/157 [==============================] - 0s 3ms/step - loss: 1.8668 - accuracy: 0.3582
Epoch 5/50
157/157 [==============================] - 0s 3ms/step - loss: 1.8446 - accuracy: 0.3592
Epoch 6/50
157/157 [==============================] - 1s 3ms/step - loss: 1.8322 - accuracy: 0.3616
Epoch 7/50
157/157 [==============================] - 1s 4ms/step - loss: 1.8293 - accuracy: 0.3630
Epoch 8/50
157/157 [==============================] - 1s 4ms/step - loss: 1.8164 - accuracy: 0.3696
Epoch 9/50
157/157 [==============================] - 1s 4ms/step - loss: 1.8082 - accuracy: 0.3722
Epoch 10/50
157/157 [==============================] - 1s 3ms/step - loss: 1.8089 - accuracy: 0.3740

In [285]:
preds = red_neuronal.predict(x_test)

18/18 [==============================] - 0s 3ms/step


In [286]:
y_pred = np.zeros(len(preds))
for i in range(len(preds)):
    y_pred[i] = np.argmax(preds[i])

y_pred

array([1., 1., 5., 1., 5., 1., 1., 5., 1., 1., 1., 1., 1., 1., 1., 1., 8.,
       1., 1., 1., 1., 1., 1., 1., 1., 5., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 5., 8., 8., 1., 1., 1., 1., 1., 1., 5., 1., 8., 1., 1., 1., 1.,
       1., 1., 5., 1., 1., 1., 1., 1., 5., 1., 1., 5., 1., 1., 5., 1., 1.,
       1., 1., 1., 1., 1., 5., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 5., 1., 1., 5., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 5.,
       1., 1., 5., 1., 1., 1., 1., 1., 8., 1., 1., 5., 5., 1., 1., 1., 5.,
       5., 5., 1., 5., 1., 1., 1., 5., 1., 1., 1., 5., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 5., 1., 5., 5., 5., 1., 1., 1., 8., 5.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 5., 5., 1., 1., 8., 1., 1., 5., 1., 1., 1., 1., 5., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 5., 1., 1., 1., 5., 1., 5., 1.,
       1., 8., 1., 1., 1., 1., 1., 1., 1., 5., 5., 1., 8., 5., 1., 5., 1.,
       5., 1., 1., 1., 1.

In [288]:
aciertos = 0
for i in range(len(preds)):
    if y_test[i] == y_pred[i]:
        aciertos += 1
accuracy = (aciertos/(len(y_test)))*100
print(f'Accuracy del modelo: {accuracy}%')

Accuracy del modelo: 37.27598566308244%
